In [3]:
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec

# numpy
import numpy

# random
from random import shuffle

# classifier
from sklearn.linear_model import LogisticRegression
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
import re
import matplotlib.pyplot as plt

In [4]:
docs = {}

class LabeledLineSentence(object):
    def __init__(self, sources):
        self.sources = sources

        flipped = {}

        # make sure that keys are unique
        for key, value in sources.items():
            if value not in flipped:
                flipped[value] = [key]
            else:
                raise Exception('Non-unique prefix encountered')

    def __iter__(self):
        for source, prefix in self.sources.items():
            with utils.smart_open(source) as fin:
                for item_no, line in enumerate(fin):
                    #item, line = line.split(" : ")
                    yield LabeledSentence(utils.to_unicode(line).split(), [prefix + '_%s' % item_no])
                    #yield LabeledSentence(utils.to_unicode(line).split(), [item])

    def to_array(self):
        self.sentences = []
        for source, prefix in self.sources.items():
            with utils.smart_open(source) as fin:
                for item_no, line in enumerate(fin):
                    #item, line = line.split(" : ")
                    docs[item_no] = line
                    self.sentences.append(LabeledSentence(utils.to_unicode(line).split(), [prefix + '_%s' % item_no]))
                    #self.sentences.append(LabeledSentence(utils.to_unicode(line).split(), [item]))
        return self.sentences

    def sentences_perm(self):
        shuffle(self.sentences)
        return self.sentences

In [32]:
sources = {"~/Desktop/tweet_dataset.txt": "SENT"}# , "../data/wonderland.txt": "AL"}
sentences = LabeledLineSentence(sources)

model = Doc2Vec(min_count=1, window=7, size=100, sample=1e-4, negative=5, workers=8)

model.build_vocab(sentences.to_array())

print model

Doc2Vec(dm/m,d100,n5,w7,s0.0001,t8)


In [33]:
for epoch in range(20):
    print("E: ", epoch)
    model.train(sentences.sentences_perm() )
    # model.alpha -= 0.002  # decrease the learning rate
    # model.min_alpha = model.alpha  # fix the learning rate, no decay

('E: ', 0)
('E: ', 1)
('E: ', 2)
('E: ', 3)
('E: ', 4)
('E: ', 5)
('E: ', 6)
('E: ', 7)
('E: ', 8)
('E: ', 9)
('E: ', 10)
('E: ', 11)
('E: ', 12)
('E: ', 13)
('E: ', 14)
('E: ', 15)
('E: ', 16)
('E: ', 17)
('E: ', 18)
('E: ', 19)


In [31]:
model.save('../data/imdb.d2v')

import pprint
pp = pprint.PrettyPrinter(indent=4)
# print("0:")
# pp.pprint(model.docvecs.most_similar(["SENT_0"]))
# print("195:")
# pp.pprint(model.docvecs.most_similar(["SENT_195"]))
# print("226:")
# pp.pprint(model.docvecs.most_similar(["SENT_226"]))

md = 5000

X = numpy.empty(shape=(md,100))
for val in docs:
    X[val] = model.docvecs[val]

# tsne = TSNE(n_components=2)
# X_tsne = tsne.fit_transform(X)
# import ipdb; ipdb.set_trace()
kmeans = KMeans(n_clusters=5,random_state=0).fit(X)

# plt.scatter( X_tsne[:, 0], X_tsne[:, 1] )
# for val in range(251):
#     labelv.append("SENT_" + str(val))

from collections import defaultdict
results = defaultdict(list)

ValueError: could not broadcast input array from shape (400) into shape (100)

In [8]:
for i, k_label in enumerate(kmeans.labels_):
    results[k_label].append(i)

for i, v in results.items():
    print i, v
    print '\n'

dist_to_cent_1 = []
for i in range(len(X)):
        dist_to_cent_1.append(numpy.linalg.norm(X[i]-kmeans.cluster_centers_[0]))

0 [8, 60, 117, 144, 149, 175, 185, 227, 301, 302, 303, 304, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 319, 320, 323, 326, 327, 328, 329, 330, 332, 333, 334, 335, 336, 337, 338, 339, 340, 343, 344, 346, 347, 360, 366, 393, 394, 397, 399, 415, 445, 447, 448, 449, 450, 451, 498, 499, 502, 524, 548, 549, 550, 551, 552, 553, 554, 570, 587, 600, 601, 602, 603, 605, 607, 654, 656, 657, 658, 659, 660, 706, 707, 708, 710, 711, 712, 725, 741, 761, 762, 766, 770, 786, 789, 808, 809, 810, 811, 833, 857, 858, 860, 861, 862, 863, 909, 911, 912, 913, 914, 952, 961, 962, 963, 965, 966, 1001, 1002, 1003, 1005, 1010, 1019, 1020, 1021, 1025, 1026, 1027, 1029, 1033, 1035, 1038, 1042, 1045, 1049, 1054, 1055, 1056, 1057, 1074, 1078, 1079, 1082, 1087, 1090, 1092, 1094, 1095, 1097, 1105, 1112, 1118, 1127, 1134, 1158, 1162, 1190, 1200, 1201, 1205, 1206, 1207, 1209, 1211, 1212, 1213, 1218, 1225, 1230, 1235, 1236, 1237, 1238, 1245, 1248, 1250, 1253, 1254, 1257, 1259, 1260, 1264, 1267, 1268, 126

In [9]:
#import ipdb; ipdb.set_trace()
index = dist_to_cent_1.index(min(dist_to_cent_1))

print docs[index], index

# 2
dist_to_cent_1 = []
for i in range(len(X)):
        dist_to_cent_1.append(numpy.linalg.norm(X[i]-kmeans.cluster_centers_[1]))

#import ipdb; ipdb.set_trace()
index = dist_to_cent_1.index(min(dist_to_cent_1))

print docs[index], index

# 3
dist_to_cent_1 = []
for i in range(len(X)):
        dist_to_cent_1.append(numpy.linalg.norm(X[i]-kmeans.cluster_centers_[2]))

#import ipdb; ipdb.set_trace()
index = dist_to_cent_1.index(min(dist_to_cent_1))

print docs[index], index

# 4
dist_to_cent_1 = []
for i in range(len(X)):
        dist_to_cent_1.append(numpy.linalg.norm(X[i]-kmeans.cluster_centers_[3]))

# import ipdb; ipdb.set_trace()
index = dist_to_cent_1.index(min(dist_to_cent_1))

print docs[index], index

# 5
dist_to_cent_1 = []
for i in range(len(X)):
        dist_to_cent_1.append(numpy.linalg.norm(X[i]-kmeans.cluster_centers_[4]))

#import ipdb; ipdb.set_trace()
index = dist_to_cent_1.index(min(dist_to_cent_1))

print docs[index], index

Jv aller dodo j'sui ko 👎😴
811
My heart and prayers are with Japan.
3480
Tweeted so much today!
4800
ニュース:Charlie Hebdo attack in Paris, manhunt on for attackers
87
is sending positive energy to Japan.
3398


In [24]:
summaries = []
for i, val in results.items():
    count_0 = 0
    count_1 = 0
    count_2 = 0
    count_3 = 0
    count_4 = 0
    summary_paris = dict()
    for v in val:
        if v <= 1000:
            count_0 += 1
        elif v > 1000 and v <= 2000:
            count_1 += 1
        elif v > 2000 and v <= 3000:
            count_2 += 1
        elif v > 3000 and v <= 4000:
            count_3 += 1
        else:
            count_4 += 1
    summary_paris[0] = count_0
    summary_paris[1] = count_1
    summary_paris[2] = count_2
    summary_paris[3] = count_3
    summary_paris[4] = count_4
    summaries.append(summary_paris)
    print summary_paris

{0: 119, 1: 235, 2: 189, 3: 87, 4: 99}
{0: 329, 1: 714, 2: 211, 3: 200, 4: 205}
{0: 114, 1: 26, 2: 491, 3: 54, 4: 478}
{0: 420, 1: 8, 2: 49, 3: 64, 4: 156}
{0: 19, 1: 17, 2: 60, 3: 595, 4: 61}


In [25]:
pp.pprint(summaries)

[   {   0: 119, 1: 235, 2: 189, 3: 87, 4: 99},
    {   0: 329, 1: 714, 2: 211, 3: 200, 4: 205},
    {   0: 114, 1: 26, 2: 491, 3: 54, 4: 478},
    {   0: 420, 1: 8, 2: 49, 3: 64, 4: 156},
    {   0: 19, 1: 17, 2: 60, 3: 595, 4: 61}]


In [26]:
summary_paris = {}
count_0 = 0
count_1 = 0
count_2 = 0
count_3 = 0
count_4 = 0
for i in range(1001):
    if kmeans.labels_[i] == 0:
        count_0 += 1
    elif kmeans.labels_[i] == 1:
        count_1 += 1
    elif kmeans.labels_[i] == 2:
        count_2 += 1
    elif kmeans.labels_[i] == 3:
        count_3 += 1
    else:
        count_4 += 1
summary_paris[0] = count_0
summary_paris[1] = count_1
summary_paris[2] = count_2
summary_paris[3] = count_3
summary_paris[4] = count_4
print summary_paris

{0: 119, 1: 329, 2: 114, 3: 420, 4: 19}


In [27]:
summary_asia = {}
count_0 = 0
count_1 = 0
count_2 = 0
count_3 = 0
count_4 = 0
for i in range(1001,2001):
    if kmeans.labels_[i] == 0:
        count_0 += 1
    elif kmeans.labels_[i] == 1:
        count_1 += 1
    elif kmeans.labels_[i] == 2:
        count_2 += 1
    elif kmeans.labels_[i] == 3:
        count_3 += 1
    else:
        count_4 += 1
summary_asia[0] = count_0
summary_asia[1] = count_1
summary_asia[2] = count_2
summary_asia[3] = count_3
summary_asia[4] = count_4
print summary_asia

{0: 235, 1: 714, 2: 26, 3: 8, 4: 17}


In [28]:
summary_irene = {}
count_0 = 0
count_1 = 0
count_2 = 0
count_3 = 0
count_4 = 0
for i in range(2001,3001):
    if kmeans.labels_[i] == 0:
        count_0 += 1
    elif kmeans.labels_[i] == 1:
        count_1 += 1
    elif kmeans.labels_[i] == 2:
        count_2 += 1
    elif kmeans.labels_[i] == 3:
        count_3 += 1
    else:
        count_4 += 1
summary_irene[0] = count_0
summary_irene[1] = count_1
summary_irene[2] = count_2
summary_irene[3] = count_3
summary_irene[4] = count_4
print summary_irene

{0: 189, 1: 211, 2: 491, 3: 49, 4: 60}


In [29]:
summary_japan = {}
count_0 = 0
count_1 = 0
count_2 = 0
count_3 = 0
count_4 = 0
for i in range(3001,4001):
    if kmeans.labels_[i] == 0:
        count_0 += 1
    elif kmeans.labels_[i] == 1:
        count_1 += 1
    elif kmeans.labels_[i] == 2:
        count_2 += 1
    elif kmeans.labels_[i] == 3:
        count_3 += 1
    else:
        count_4 += 1
summary_japan[0] = count_0
summary_japan[1] = count_1
summary_japan[2] = count_2
summary_japan[3] = count_3
summary_japan[4] = count_4
print summary_japan

{0: 87, 1: 200, 2: 54, 3: 64, 4: 595}


In [30]:
summary_london = {}
count_0 = 0
count_1 = 0
count_2 = 0
count_3 = 0
count_4 = 0
for i in range(4001,5000):
    if kmeans.labels_[i] == 0:
        count_0 += 1
    elif kmeans.labels_[i] == 1:
        count_1 += 1
    elif kmeans.labels_[i] == 2:
        count_2 += 1
    elif kmeans.labels_[i] == 3:
        count_3 += 1
    else:
        count_4 += 1
summary_london[0] = count_0
summary_london[1] = count_1
summary_london[2] = count_2
summary_london[3] = count_3
summary_london[4] = count_4
print summary_london

{0: 99, 1: 205, 2: 478, 3: 156, 4: 61}
